In [467]:
import pandas as pd
import numpy as np
from sklearn.linear_model import *
from sklearn.datasets import *
from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.feature_extraction.text import *
from sklearn.naive_bayes import *
from sklearn.linear_model import *
from sklearn.ensemble import *
from operator import itemgetter, attrgetter, methodcaller

def topk_acc(predictions, test, k=3):
    pred = pd.DataFrame(predictions, columns=nb.classes_, index=test.index)
    hits = 0
    for i, row in pred.iterrows():
        row = zip(row.index, list(row))
        topk, prob = zip(*sorted(row, key=itemgetter(1), reverse=True)[:k])
        if test['labelx'][i] in topk: hits += 1
    return float(hits) / pred.shape[0]

In [652]:
df    = fetch_20newsgroups()
vec   = CountVectorizer(stop_words='english', max_features=500)
X     = vec.fit_transform(df.data)
vocab = vec.get_feature_names()
len(vocab)

500

In [653]:
data  = pd.DataFrame(X.todense(), columns=vocab)
data['labelx']  = [ df.target_names[i] for i in df.target ]
data['ranking'] = [ [ int(j == i) for j in range(len(df.target_names)) ] for i in df.target ]

test_i = np.random.choice(data.index, size=int(data.shape[0]*.1), replace=False)
test, train = data[np.in1d(data.index,test_i)], data[~np.in1d(data.index,test_i)]

## Benchmark

In [654]:
nb = MultinomialNB(alpha=1)
nb.fit(train[vocab], train['labelx'])

p1 = nb.predict_proba(test[vocab])
topk_acc(p1, test, k=3), coverage_error(test['ranking'].tolist(), p1)

(0.7957559681697612, 2.8001768346595934)

In [655]:
nb = RandomForestClassifier(n_estimators=100, min_weight_fraction_leaf=.0005, n_jobs=-1)
nb.fit(train[vocab], train['labelx'])

p2 = nb.predict_proba(test[vocab])
topk_acc(p2, test, k=3), coverage_error(test['ranking'].tolist(), p2)

(0.8585322723253758, 2.2307692307692308)

In [656]:
nb = SGDClassifier(alpha=.01, loss='log', penalty='elasticnet', l1_ratio=.0, n_jobs=-1)
nb.fit(train[vocab], train['labelx'])

p3 = nb.predict_proba(test[vocab])
topk_acc(p3, test, k=3), coverage_error(test['ranking'].tolist(), p3)

(0.8160919540229885, 2.4350132625994694)

## Naive Bayes

In [668]:
xnb = MyNB()
xnb.fit(train[vocab], train['labelx'])

%time p = xnb.predict(test[vocab])
topk_acc(p, test, k=3), coverage_error(test['ranking'].tolist(), p)

CPU times: user 2.8 s, sys: 8.1 ms, total: 2.81 s
Wall time: 2.82 s


(0.14588859416445624, 10.426171529619806)

In [667]:
from collections import defaultdict

class MyNB:
    def __init__(self):
        self.features = []
        self.counts   = { 
            'n': 0, 
            'y': defaultdict(lambda: 0),
            'x': defaultdict(lambda: defaultdict(lambda: 0)),
            'x_y': defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: 0)))
        }

    def predict(self, data, classes=[]):
        totals = []
        for row in data.values:
            probs = [ self.probs["{}_{}".format('y', c)] for c in classes ]
            for i, c in enumerate(classes):
                probs[i] += np.sum([ self.probs[c][j][v] for j,v in enumerate(row) ])
            totals.append(probs)
        return totals
        
    def fit(self, data, labels, report_k=None):
        for i, row in enumerate(data.values):
            if report_k and self.counts['n'] % report_k == 0: print self.counts['n']
            self.counts['n'] += 1
            self.counts[ labels[i] ][ 0 ] += 1
            for j, v in enumerate(row):
                self.counts[ '_x_cnt_' ][ j ][ v ] += 1
                self.counts[ labels[i] ][ j ][ v ] += 1
        
        for y, x in self.counts[ 'x_y' ].iteritems():
            self.probs[y][0] = np.log(self._prior(y))
            for j, posterior in x.iteritems():
                self.probs[y][j] = defauctdict(lambda x: np.log(self._smoothing(y, j))) 
                for value, _ in posterior.iteritems():
                    self.probs[y][j][value] = np.log(self._posterior(y, name, value))
        
    def _posterior(self, y, f, v):
        return float( self.counts['x_y'][y][f][v] + 1 ) / ( len(self.counts['x'][f]) + self.counts['y'][y] ) 
    
    def _smoothing(self, y, f ):
        return 1.0 / ( len(self.counts['x'][f]) + self.counts['y'][y] )

    def _prior(self, y):
        return float( self.counts['y'][y] ) / self.counts['n']
    

## Streaming test

In [597]:
def trainx(model, dataset, labels, classes):
    for i, row in enumerate(dataset):
        nb.partial_fit([row],[labels[i]], classes=classes)
    return nb

nb   = MultinomialNB(alpha=1)
vals = train[vocab].values
%time trainx(nb, vals, train['labelx'].values, df.target_names)

%time p1 = nb.predict_proba(test[vocab])
topk_acc(p1, test, k=3), coverage_error(test['ranking'].tolist(), p1)

CPU times: user 7.31 s, sys: 56.2 ms, total: 7.37 s
Wall time: 7.4 s
CPU times: user 14.1 ms, sys: 2.52 ms, total: 16.6 ms
Wall time: 11.2 ms


(0.8488063660477454, 2.6065428824049515)